In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('./output/train_hypothesis_evidences.csv')
test_df

,hypothesis,evidences,label
0,Patients with cytologically confirmed breast c...,"[""Inclusion Criteria:"", "" Histologically or c...",Contradiction
1,Women with leptomeningeal metastases are unfor...,"["" Exclusion Criteria"", "" Patients with know...",Contradiction
2,Cohort 1 of NCT02513472 produced better Object...,"[""Outcome Measurement: "", "" Objective Respons...",Entailment
3,NCT01237327 and NCT00030823 do not have any ov...,"[""Inclusion Criteria:"", "" Previous participat...",Entailment
4,More than 2% of patients in NCT00129376 experi...,"[""Adverse Events 1:"", "" Acute Pharyngitis * 1...",Contradiction
...,...,...,...
715,NCT00696072 uses Overall Survival (OS) as outc...,"[""Outcome Measurement: "", "" Proportion of Par...",Contradiction
716,NCT00320541 recorded more cardiac related adve...,"[""Adverse Events 1:"", "" Total: 17/50 (34.00%)...",Contradiction
717,There were 2 cases of Angina in NCT02370238.,"[""Adverse Events 1:"", "" Total: 13/61 (21.31%)...",Contradiction
718,NCT00645333 records several central nervous sy...,"["" Autoimmune disorder *1/30 (3.33%)"", "" Imm...",Contradiction


In [3]:
hypothesis_lst=test_df['hypothesis'].values.tolist()
len(hypothesis_lst)

720

In [4]:
evidence_lst=test_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

720

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

720

In [6]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [7]:
# text_tok=AutoTokenizer.from_pretrained('bert-base-uncased')
# text_clf=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
#roberta-base
text_tok=AutoTokenizer.from_pretrained('roberta-base')
text_clf=AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=2)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [8]:
class InputSequence:
    
    def __init__(self,tok,l_text,l_text2,l_label,batch_size=64,gpu=True):
        
        self.data_len=len(l_text)
        self.data_idx=[i for i in range(self.data_len)]
        self.texts=tok(l_text,l_text2,padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label=np.array(l_label)
        print('tokenize done')
        
        self.batch_size=batch_size
        self.gpu=gpu
        
    def on_epoch_end(self):
        random.shuffle(self.data_idx)
        
    def __getitem__(self,i):
        start=i*self.batch_size
        batch_idx=self.data_idx[start:min(start+self.batch_size,self.data_len)]
        
        return_texts=dict([(k,self.texts[k][batch_idx]) for k in self.texts])
        return_labels=torch.from_numpy(
            self.l_label[batch_idx].astype(np.int64)
        )
        
        if self.gpu:
            return_texts=dict([(k,return_texts[k].cuda()) for k in return_texts])
            return_labels=return_labels.cuda()
        
        return return_texts,return_labels
    
    def __len__(self):
        return math.ceil(1.0*self.data_len/self.batch_size)
    

In [9]:
testing_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

tokenize done


In [10]:
scores=[]
# model_names=['bert-base-uncased']+[
#     './output/clf_models/bert-base-uncased_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#roberta-base
model_names=['roberta-base']+[
    './output/clf_models/roberta-base_epoch_{}.pt'.format(format(epoch,'05d'))
    for epoch in range(10)
]
for model_name in model_names:
    scores.append([])
    clf=AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
    with torch.no_grad():
        for batch in range(len(testing_data)):
            batch_texts,batch_labels=testing_data[batch]
            scores[-1].append(F.softmax(clf(**batch_texts).logits,dim=1).detach().cpu().numpy())
            print('model:',model_name,'batch:',batch,end='\r')
    scores[-1]=np.concatenate(scores[-1],axis=0)
    clf.cpu()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [11]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for epoch in range(len(scores)):
    y_prob=scores[epoch][:,1]
    y_pred=[1 if a>0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch==0 else epoch,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['epoch','AVG_PREC','F1','PREC','REC','ACC'])

,epoch,AVG_PREC,F1,PREC,REC,ACC
0,pretrained,0.491515,0.661710,0.494444,1.000000,0.494444
1,1,0.563004,0.658467,0.496434,0.977528,0.498611
2,2,0.576840,0.038462,0.875000,0.019663,0.513889
3,3,0.585947,0.421818,0.597938,0.325843,0.558333
4,4,0.600798,0.407547,0.620690,0.303371,0.563889
5,5,0.623830,0.675024,0.513950,0.983146,0.531944
6,6,0.671782,0.618557,0.650155,0.589888,0.640278
7,7,0.753473,0.714579,0.563107,0.977528,0.613889
8,8,0.827803,0.766055,0.647287,0.938202,0.716667
9,9,0.868962,0.829517,0.758140,0.915730,0.813889
